In [1]:
import logging
import platform
import re
import socket
import sys
import warnings

import psutil
from marshmallow.warnings import ChangedInMarshmallow3Warning

logger = logging.getLogger(__name__)


def _check_python_version():
    """Check for Python version 3.5+."""
    if sys.version_info < (3, 5):
        raise Exception('Qiskit requires Python version 3.5 or greater.')


def _filter_deprecation_warnings():
    """Apply filters to deprecation warnings.
    Force the `DeprecationWarning` warnings to be displayed for the qiskit
    module, overriding the system configuration as they are ignored by default
    [1] for end-users. Additionally, silence the `ChangedInMarshmallow3Warning`
    messages.
    TODO: on Python 3.7, this might not be needed due to PEP-0565 [2].
    [1] https://docs.python.org/3/library/warnings.html#default-warning-filters
    [2] https://www.python.org/dev/peps/pep-0565/
    """
    deprecation_filter = ('always', None, DeprecationWarning,
                          re.compile(r'^qiskit\.*', re.UNICODE), 0)

    # Instead of using warnings.simple_filter() directly, the internal
    # _add_filter() function is used for being able to match against the
    # module.
    try:
        warnings._add_filter(*deprecation_filter, append=False)
    except AttributeError:
        # ._add_filter is internal and not available in some Python versions.
        pass

    # Add a filter for ignoring ChangedInMarshmallow3Warning, as we depend on
    # marhsmallow 2 explicitly. 2.17.0 introduced new deprecation warnings that
    # are useful for eventually migrating, but too verbose for our purposes.
    warnings.simplefilter('ignore', category=ChangedInMarshmallow3Warning)


_check_python_version()
_filter_deprecation_warnings()

In [2]:
import copy
from openfermion.hamiltonians import MolecularData
import numpy as np
from itertools import combinations
from pprint import pprint
from openfermionpsi4 import run_psi4
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
import xlrd
from scipy.optimize import minimize
from openfermion.ops import FermionOperator
arrange=list(combinations([0,1,2,3,4,5,6,7],6))
matrixa1=np.mat=[[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]

matrix=np.transpose(matrixa1)

In [3]:
def allzfun(COM,gate):
    no=-1
    S=-1
    ss=len(COM)
    allz=[[]]*ss 
    for n in range(0,ss):
        no=no+1   
        SG=gate[n].split()
        gateno=len(SG)
        jud=0
        for X in range (0,gateno):
            if ord(SG[X][0]) ==90:
                jud=jud+1
        
        if jud==gateno:
            S=S+1  
            allz[S]=no
        
    while [] in allz:
        allz.remove([])     
    return allz

In [4]:
def unperturbate(bktrstate):
    
    trenergy=0
    allzno=len(allz)
    for nn in range (0,allzno):
        a=gate[allz[nn]].split('Z')
        a.remove('')
        sign=0
        lena=len(a)
        for nnn in range (0,lena):
            noz=int(a[nnn])
            if bktrstate[noz]==1:
                sign=sign+1
        
        trenergy=(-1)**sign*COM[allz[nn]]+trenergy
    return trenergy
def gatexyzandqubit():
    ss=len(COM)
    gatexyz=[[]]*ss
    gatequbit=[[]]*ss
    for n in range (0,ss):
    
        gatesplit=gate[n].split()
        gatexyz[n]=[0]*len(gatesplit)
        gatequbit[n]=[0]*len(gatesplit)
        for nn in range (0,len(gatesplit)):
                gatexyz[n][nn]=ord(gatesplit[nn][0])
                gatequbit[n][nn]=int(gatesplit[nn][1:])  
    return (gatexyz,gatequbit)

def intialstate():
    trenergy0=10
    for n in range (0,len(arrange)):
        trail=arrange[n]
        trstate=[0]*16
        for a in range (0,len(trail)):
            trstate[trail[a]]=1
        bktrstate =matrix.dot(trstate)%2
        trenergy=unperturbate(bktrstate).real
        
        
        if trenergy<=trenergy0:
            trenergy0=trenergy
            bktrstate0=matrix.dot(trstate)%2  
    print(trenergy0,bktrstate0)
    return (trenergy0,bktrstate0)


In [5]:
def preucc():
    ss=len(COM)
    coefficientduexyz=[[]]*ss

    for n in range (0,ss):
        coefficientduexyz[n]=[[]]*len(gatexyz[n])
        for nn in range (0,len(gatexyz[n])):
    
            if bktrstate0[gatequbit[n][nn]]==1:
                if gatexyz[n][nn]==88:    
                    coefficientduexyz[n][nn]=1
                elif gatexyz[n][nn]==89:
                    coefficientduexyz[n][nn]=-1j
                elif gatexyz[n][nn]==90:
                    coefficientduexyz[n][nn]=-1
            elif bktrstate0[gatequbit[n][nn]]==0:
                if gatexyz[n][nn]==88:
                    coefficientduexyz[n][nn]=1
                elif gatexyz[n][nn]==89:
                    coefficientduexyz[n][nn]=1j
                elif gatexyz[n][nn]==90:
                    coefficientduexyz[n][nn]=1   
    changequbit=copy.deepcopy(gatequbit)
    gatexy=copy.deepcopy(gatexyz)
    for n in range (0,ss):
        for nn in range (0,len(gatexyz[n])):
            if gatexyz[n][nn]==90:
                gatexy[n][nn]=[]
                changequbit[n][nn]=[]
################changequbit 
    for n in range (0,ss):
        changequbit[n]=[x for x in  changequbit[n] if x!=[]]
        gatexy[n]=[x for x in  gatexy[n] if x!=[]]
    cofxyz=[1]*ss
    for n in range (0,ss):
        for nn in coefficientduexyz[n]:
            cofxyz[n]*=nn
    cofstate=[1]*ss 
    for n in range (1,ss):
        if len(changequbit[n])!=0:      
            if bktrstate0[changequbit[n][0]]==0:
                cofstate[n]=-1
    cof=[[]]*ss
    for n in range (0,ss):
        cof[n]=cofxyz[n]*cofstate[n]
    co=[[]]*ss
    for n in range (0,ss):
        co[n]=COM[n]*cof[n]
    perturbastate=[[]]*ss
    for n in range (0,ss):
        changestate=np.array([0]*16) 
        for nn in range (0,len(changequbit[n])):
            changestate[changequbit[n][nn]]=1
        perturbastate[n]=(bktrstate0+changestate)%2
    reduce=[[]]
    reduce[0]=[co[0]]
    reducestate=[[]]
    reducestate[0]=perturbastate[0]
    reducequbit=[[]]
    reducequbit[0]=[changequbit[0]]
    reducegate=[[]]
    reducegate[0]=[gatexy[0]]
    for n in range(1,ss):
        a=0
        for nn in range (0,len(reducestate)):
            if np.linalg.norm(perturbastate[n]-reducestate[nn])==0:
                reduce[nn].append(co[n])   
                a=1
        if a==0:
            reducestate.append(perturbastate[n])
            reduce.append([co[n]])
            reducegate.append([gatexy[n]])
            reducequbit.append(changequbit[n])
           
    reducecof=[[]]*len(reduce)
    for n in range (0,len(reduce)):
        reducecof[n]=sum(reduce[n])
            
######
    deltaenergy=[[]]*len(reducestate)
    for n in range (0,len(reducestate)):
        chstate=reducestate[n]
        trenergy=unperturbate(chstate)
        deltaenergy[n]=trenergy0-trenergy
    UCCC=[[]]*len(deltaenergy)
    for n in range (0,len(deltaenergy)):
        if deltaenergy[n] !=0:
            UCCC[n]=   reducecof[n]/ deltaenergy[n]
    UCCCO=[]
    UCQUBIT=[]
    for i in range (0,len(UCCC)):
        if UCCC[i] !=0 and UCCC[i] !=[]:
            UCCCO.append(UCCC[i])
            UCQUBIT.append(reducequbit[i])
    reducequbit=UCQUBIT      
    return (UCCCO,reducequbit)

In [6]:
from qiskit import IBMQ 
from qiskit import execute, BasicAer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from math import pi
from qiskit.tools.visualization import matplotlib_circuit_drawer as drawer, qx_color_scheme

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
qr = QuantumRegister(14)
cr = ClassicalRegister(14)
Cix= QuantumCircuit(qr, cr)

In [7]:
def ucc(UCCCO):


    ans=[]
    for n in range (1,len(COM)):
        
        qr = QuantumRegister(8)
        cr = ClassicalRegister(8)
        Cix= QuantumCircuit(qr, cr)
        initial=list(bktrstate0)
        for i in range(len(initial)):
            if initial[i]==1:
                Cix.x(qr[i])
#################################  
        
        for i in range(0,len(UCCCO)):
            
            for j in range(1,len(reducequbit[i])):
                Cix.h(qr[(reducequbit[i][j])])
            Cix.rx(-0.5*pi,qr[reducequbit[i][0]])
            for k in range(len(reducequbit[i])-1):
                Cix.cx(qr[reducequbit[i][k]],qr[reducequbit[i][k+1]])
            Cix.rz((UCCCO[i].real)*2,qr[reducequbit[i][-1]])
            for s in range(len(reducequbit[i])-1,0,-1):
                Cix.cx(qr[reducequbit[i][s-1]],qr[reducequbit[i][s]])
            for j in range(1,len(reducequbit[i])):
                Cix.h(qr[reducequbit[i][j]])
            Cix.rx(0.5*pi,qr[reducequbit[i][0]])
                
 #############################               
              
        for j in range(len(gatequbit[n])):
            if gatexyz[n][j]==88:
                Cix.h(qr[gatequbit[n][j]])
            if gatexyz[n][j]==89:
                Cix.rx(-0.5*pi,qr[gatequbit[n][j]])
            if gatexyz[n][j]==90:
                continue
            
    
        backend = BasicAer.get_backend('statevector_simulator')
        job = execute(Cix, backend)
        result = job.result().get_statevector(Cix, decimals=None)
        result=list(result)
            
        
        odd=0
        even=0
       
        for s in range(0,len(result)):
            aaaa=0
            aa=[0]*8            
            AA=np.binary_repr(s)
            
            for nn in range (0,len(AA)):
                aa[nn]=int(AA[len(AA)-nn-1])
            for aaa in range (0,len(gatequbit[n])):
                aaaa+=aa[gatequbit[n][aaa]]
            if aaaa%2==0:
                even+=abs(result[s])**2
            else:
                odd+=abs(result[s])**2
          
        ans.append((even-odd)*COM[n])
        
        
        ccenergy=(sum(ans)+COM[0]).real
    print(ccenergy,UCCCO)
    return  ccenergy

In [8]:
asd= xlrd.open_workbook("8o1.xlsx")
single=asd.sheets()[0]
b=single.col_values(0)
aa=[]
for a in range (0,len(b)):
    aa.append(single.row_values(a))
single=aa
asdf= xlrd.open_workbook("8o2.xlsx")
double=asdf.sheets()[0]
b=double.col_values(0)
aa=[]
for a in range (0,len(b)):
    aa.append(double.row_values(a))
double=aa


In [9]:

gsenergy=[]
for aaa in range (4,len(double[0])):
   
    p=int(single[0][0])
    q=int(single[0][1])
    cof=float(single[0][aaa-2])
    single_term = FermionOperator(((p, 1), (q, 0)) ,cof)
    for n in range (1,len(single)):
        p=int(single[n][0])
        q=int(single[n][1])
        cof=single[n][aaa-2]
        single_term += FermionOperator(((p, 1), (q, 0)) ,cof)
    jwsingle=jordan_wigner( single_term )
    p=int(double[0][0])
    q=int(double[0][1])
    r=int(double[0][2])
    s=int(double[0][3])
    cof=double[0][aaa]
    double_term = FermionOperator(((p, 1), (q, 1) ,(r, 0), (s, 0)),cof)
    for n in range (1,len(double)):
        p=int(double[n][0])
        q=int(double[n][1])
        r=int(double[n][2])
        s=int(double[n][3])
        cof=double[n][aaa]
        double_term += FermionOperator(((p, 1), (q, 1) ,(r, 0), (s, 0)),cof)
    jwdouble=jordan_wigner( double_term )
    ham=jwsingle+jwdouble
    b=ham
    b=str(b)
    strb=b.split('+\n')
    ss=len(strb)
    COM=[[]]*ss
    gate=[]
    CC=[]
    for n in range (0,ss):
        BB=strb[n]
        CC.append(BB.split('[')[0])
        CC[n]=complex(CC[n].strip('(').split(')')[0])
        gate.append(BB.split('[')[1].split(']')[0])
COM=CC 
allz=allzfun(COM,gate)
(gatexyz,gatequbit)=gatexyzandqubit()
(trenergy0,bktrstate0)=intialstate()
(UCCCO,reducequbit)=preucc()    


-12.782734810541445 [1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]


In [11]:
UCCCO

[(-0.046266551962910386-0j),
 (5.012990489693049e-08-0j),
 (-5.0129904932936655e-08-0j),
 (-0.04772192698773726-0j),
 (-0.03237780383095615-0j)]

In [12]:
reducequbit

[[0, 1, 6, 7], [2, 6], [3, 7], [2, 3, 6, 7], [4, 5, 6, 7]]

In [13]:
# NEW PART
UCCCO_mod=[]
for i in range(len(UCCCO)):
    UCCCO_mod.append(UCCCO[i].real)
pair = [p for p in zip(reducequbit, UCCCO_mod)]
for i in range(len(reducequbit)):
    reducequbit[i]=sorted(pair,key=lambda x:abs(x[1]),reverse=True)[i][0]
    UCCCO[i]=sorted(pair,key=lambda x:abs(x[1]),reverse=True)[i][1]

In [14]:
UCCCO

[-0.04772192698773726,
 -0.046266551962910386,
 -0.03237780383095615,
 -5.0129904932936655e-08,
 5.012990489693049e-08]

In [15]:
reducequbit

[[2, 3, 6, 7], [0, 1, 6, 7], [4, 5, 6, 7], [3, 7], [2, 6]]

### Optimize Part

In [18]:
# PK_mod

number_of_parameters = 5
number_of_param_to_optimized = 3
number_of_round_to_optimize = 2
sample_input = UCCCO
totalnum=0
x_in = sample_input
delta = number_of_param_to_optimized

print('Initial x_in:',x_in)
for rnd in range(number_of_round_to_optimize):
    for i in range(number_of_parameters-delta+1):
    
        index1 = i
        index2 = i+delta-1

        def g(opt, *argv):
            fix = []
            for arg in argv: fix.append(arg)
            '''
            print('In g()')
            print(list(fix[:index1]),type(list(fix[:index1])))
            print(list(opt), type(list(opt)))
            print(list(fix[index1:]))
            print('-------------')
            '''
            x = list(fix[:index1])+list(opt)+list(fix[index1:])   
            return ucc(x)

        opt = x_in[index1:index2+1]
        fix = tuple(x_in[0:index1]+x_in[index2+1:])

        #print(opt, fix)

        res = minimize(fun = g,x0 = opt,args = fix, method = 'COBYLA')
        totalnum+=res.nfev
        #print(res)

        x_in[index1: index2+1] = res.x
        print('x_in after the #{} optimization in round {}:\n > x_in ='.format(i,rnd),x_in,end = '\n\n')
    

print('-----FINAL-------')
print('x_in =',x_in)
print('function value=',res.fun)
print('success=',res.success)
print('message:',res.message)

Initial x_in: [-0.0447499306019679, -0.0435154091313342, -0.028455152742346855, 0.003393027507908043, -0.0033999572217184373]
-12.790038814942308 [-0.0447499306019679, -0.0435154091313342, -0.028455152742346855, 0.003393027507908043, -0.0033999572217184373]
-11.755025494541584 [0.9552500693980321, -0.0435154091313342, -0.028455152742346855, 0.003393027507908043, -0.0033999572217184373]
-11.688511436730751 [-0.0447499306019679, 0.9564845908686658, -0.028455152742346855, 0.003393027507908043, -0.0033999572217184373]
-11.926318727152598 [-0.0447499306019679, -0.0435154091313342, 0.9715448472576531, 0.003393027507908043, -0.0033999572217184373]
-10.569827897992825 [-0.6392879844906, -0.6762608345654632, -0.5245980042181102, 0.003393027507908043, -0.0033999572217184373]
-12.314275377639635 [-0.342018957546284, -0.3598881218483987, -0.2765265784802285, 0.003393027507908043, -0.0033999572217184373]
-12.701055965602517 [0.18731511785843874, -0.11668648584503502, -0.08582942237279029, 0.0033930

-12.790038812821019 [-0.044726238172455586, -0.04353317001693011, -0.0284343976311499, 0.003387171718631569, -0.0033999572217184373]
-12.79003881238543 [-0.044726238172455586, -0.043521337962425674, -0.028451711248433723, 0.003341781839332839, -0.0033999572217184373]
-12.790038793657295 [-0.044726238172455586, -0.043606805798259665, -0.028378130607764066, 0.0033495978741744977, -0.0033999572217184373]
-12.790038806067148 [-0.044726238172455586, -0.0435370594295652, -0.028527022872737617, 0.003424660878267001, -0.0033999572217184373]
x_in after the #1 optimization in round 0:
 > x_in = [-0.044726238172455586, -0.0435370594295652, -0.028527022872737617, 0.003424660878267001, -0.0033999572217184373]

-12.790038806067148 [-0.044726238172455586, -0.0435370594295652, -0.028527022872737617, 0.003424660878267001, -0.0033999572217184373]
-11.92639861638975 [-0.044726238172455586, -0.0435370594295652, 0.9714729771272623, 0.003424660878267001, -0.0033999572217184373]
-12.37650411672228 [-0.044726

-12.790037575982604 [-0.04519290892020988, -0.04302961792989106, -0.027742950638895557, 0.003262640426073767, -0.0034579238204785413]
-12.790037348432488 [-0.043949986481218485, -0.04388835940328504, -0.027957437325453416, 0.003262640426073767, -0.0034579238204785413]
-12.790038486057266 [-0.04454082382421452, -0.04317918216030199, -0.028710227899952406, 0.003262640426073767, -0.0034579238204785413]
-12.790038460960487 [-0.04499474171984008, -0.04379924473191523, -0.02874699554479649, 0.003262640426073767, -0.0034579238204785413]
-12.790038690406654 [-0.044534830309987054, -0.043676720447256574, -0.02837576950052466, 0.003262640426073767, -0.0034579238204785413]
-12.79003872807988 [-0.04483196036565251, -0.04346540621602567, -0.028226541928584477, 0.003262640426073767, -0.0034579238204785413]
-12.790038789123969 [-0.04479689379840476, -0.04359475406184786, -0.028475589268663833, 0.003262640426073767, -0.0034579238204785413]
-12.79003880216909 [-0.04474524475026611, -0.04349716688580658

-12.790001147695907 [-0.0447668058395899, -0.04362981167488144, -0.02891765668331334, 0.008228102595982924, -0.00963852728279655]
-12.789976023731725 [-0.0447668058395899, -0.04362981167488144, -0.03499194848823508, -0.0007763967671219601, -0.0044391621222171634]
-12.790028777189068 [-0.0447668058395899, -0.04362981167488144, -0.029814914434752884, 0.006076852950473944, -0.000862540653375413]
-12.790022164003531 [-0.0447668058395899, -0.04362981167488144, -0.024887969143147785, 0.002711660581045813, -0.00219464954505327]
-12.790035540061151 [-0.0447668058395899, -0.04362981167488144, -0.02975706882665052, 0.00197197166746716, -0.0033377756889123628]
-12.790036176805444 [-0.0447668058395899, -0.04362981167488144, -0.02885677707583782, 0.004144820242202829, -0.00525589190829936]
-12.790037987993346 [-0.0447668058395899, -0.04362981167488144, -0.02900412090677123, 0.004143350645903074, -0.0029592716641259555]
-12.790037844534792 [-0.0447668058395899, -0.04362981167488144, -0.0275889134736

In [ ]:

x0=UCCCO
b=(-pi/2,pi/2)
bnds=[b]*len(UCCCO)
    ############optiaml
import time as TIME
t1=TIME.time()
sol = minimize(ucc,x0,method='COBYLA')
t2=TIME.time()-t1
print(sol.fun)    
gsenergy.append(sol.fun)

-12.789976062207248 [-4.77219270e-02 -4.62665520e-02 -3.23778038e-02 -5.01299049e-08
  5.01299049e-08]
-11.759051124993027 [ 9.52278073e-01 -4.62665520e-02 -3.23778038e-02 -5.01299049e-08
  5.01299049e-08]
-11.690996483027158 [-4.77219270e-02  9.53733448e-01 -3.23778038e-02 -5.01299049e-08
  5.01299049e-08]
-11.929193901450107 [-4.77219270e-02 -4.62665520e-02  9.67622196e-01 -5.01299049e-08
  5.01299049e-08]
-12.37777368077753 [-4.77219270e-02 -4.62665520e-02 -3.23778038e-02  9.99999950e-01
  5.01299049e-08]
-12.373841017120178 [-4.77219270e-02 -4.62665520e-02 -3.23778038e-02 -5.01299049e-08
  1.00000005e+00]
-10.851341739926722 [-0.61059113 -0.64629254 -0.50235166 -0.22505623 -0.2272033 ]
-12.349633005422694 [-0.32915653 -0.34627955 -0.26736473 -0.11252814 -0.11360163]
-12.70326203215767 [ 0.18442271 -0.11362251 -0.08513475 -0.02526375 -0.02550468]
-12.769302842612795 [-4.77219270e-02  3.08114682e-02 -1.30785013e-01 -5.01299049e-08
  5.01299049e-08]
-12.712992683005037 [-0.04086883  0

In [44]:
print(t2,sol.nfev)

924.2345087528229 66


In [35]:
sol.nfev

51

In [19]:
totalnum

197

In [25]:
#Reversed order
#time=760.5086395740509
#result=-10.424646908361357

#order 
#time=705.000529050827
#result=-10.369764246917264
#nfev: 51

### dictionary_test

In [ ]:
-12.782734810541445